In [10]:
# !pip install sqlite3
# !pip install time
# !pip install psutil
# !pip install os
# !pip install faker

In [13]:
import sqlite3
import time
import random
import psutil
import os
from faker import Faker
fake = Faker("pt_br")

In [19]:
# Função para monitorar memória e CPU
def monitorar_recursos():
    processo = psutil.Process(os.getpid())
    memoria = processo.memory_info().rss / (1024 * 1024)  # Memória em MB
    cpu = psutil.cpu_percent(interval=1)  # Uso de CPU em %
    return memoria, cpu

# Função para medir tempo de execução
def medir_tempo(func):
    def wrapper(*args, **kwargs):
        inicio = time.time()
        resultado = func(*args, **kwargs)
        fim = time.time()
        tempo_total = fim - inicio
        memoria, cpu = monitorar_recursos()
        print(f"Tempo de execução: {tempo_total:.2f}s | Memória usada: {memoria:.2f}MB | CPU usada: {cpu}%")
        return resultado
    return wrapper

# Contar registros 
def contar_registros(conexao):
    cursor = conexao.cursor()
    cursor.execute("SELECT COUNT(*) FROM teste;")
    resultado = cursor.fetchone()
    print(f"Total de registros: {resultado[0]}")



In [6]:
nome_banco = "sqlite_database.bd"
conexao = sqlite3.connect(nome_banco)
print(f"Banco de dados '{nome_banco}' criado.")
cursor = conexao.cursor()
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS teste (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT,
        valor INTEGER
    );
"""
)
conexao.commit()
print("Tabela 'teste' criada com sucesso.")

Banco de dados 'sqlite_database.bd' criado.
Tabela 'teste' criada com sucesso.


In [16]:
def inserir_dados(conexao, num_registros):
    cursor = conexao.cursor()
    print(f"Inserindo {num_registros} registros...")
    cursor.execute("BEGIN TRANSACTION;")
    for _ in range(num_registros):
        nome = fake.name()
        valor = random.randint(1, 1000000)
        cursor.execute("INSERT INTO teste (nome, valor) VALUES (?, ?)", (nome, valor))
    conexao.commit()
    print(f"{num_registros} registros inseridos com sucesso.")

In [21]:
# Testando o script
inserir_dados(conexao, num_registros=100000)  # Altere o valor aqui para testar o limite
conexao.close()

Inserindo 100000 registros...
100000 registros inseridos com sucesso.


In [23]:
# Testando o script
conexao = sqlite3.connect(nome_banco)
inserir_dados(conexao, num_registros=1000000)  # Altere o valor aqui para testar o limite
conexao.close()

Inserindo 1000000 registros...
1000000 registros inseridos com sucesso.


In [27]:
# Testando o script
conexao = sqlite3.connect(nome_banco)
inserir_dados(conexao, num_registros=10000000)  # Altere o valor aqui para testar o limite
conexao.close()

Inserindo 10000000 registros...
10000000 registros inseridos com sucesso.


In [30]:
conexao = sqlite3.connect(nome_banco)
contar_registros(conexao=conexao)

Total de registros: 11100000


In [35]:
select_cursor = conexao.cursor()
select_cursor.execute(
    "SELECT COUNT(nome) as contagem_nome, sum(valor) as contagem_valor from teste where nome like 'F%'"
)
resultado = select_cursor.fetchall()
print(resultado)

[(130411, 65360517318)]


In [26]:
conexao_ram = sqlite3.connect(":memory:")
print("Banco de dados em memória criado.")

Banco de dados em memória criado.
